In [1]:
import torch
from torch.autograd import grad
import torch.nn.functional as F
import torch.nn as nn

In [2]:
!pip install pandas
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import pandas as pd

iris=pd.read_csv('Iris.csv')

In [4]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
iris.Species.value_counts()

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
iris.Species = le.fit_transform(iris.Species)

In [7]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    int64  
dtypes: float64(4), int64(2)
memory usage: 7.2 KB


In [8]:
labels_np = iris.Species.to_numpy()
iris = iris.drop('Species', axis=1)
features_np = iris.to_numpy()

In [9]:
from sklearn.model_selection import train_test_split
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    features_np,labels_np , test_size=0.20, random_state=38)

In [10]:
X_train = torch.from_numpy(X_train_np).float()
Y_train = torch.from_numpy(y_train_np).long()
X_test = torch.from_numpy(X_test_np).float()
Y_test = torch.from_numpy(y_test_np).long()

In [11]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.features=X
        self.labels=y

    def __getitem__ (self,index):
        x = self.features[index]
        y= self.labels[index]
        return x, y
    def __len__(self):
        return len(self.features)

In [12]:
train_dataset=MyDataset(X_train,Y_train)
test_dataset=MyDataset(X_test,Y_test)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self,num_inputs,num_outputs):
        super().__init__()
        # 1st layer
        self.layers= nn.Sequential(
        nn.Linear(num_inputs,20),
        nn.ReLU(),
        # 2nd layer
        nn.Linear(20,20),
        nn.ReLU(),
        # 3rd layer
        nn.Linear(20,20),
        nn.ReLU(),
        nn.Linear(20,num_outputs),
        nn.Sigmoid(),
        nn.Softmax(),
        )
    def forward(self,x):
        return self.layers(x)

In [14]:
train_dataloader= torch.utils.data.DataLoader(
    train_dataset, batch_size=2, shuffle =True, num_workers=0)

In [15]:
test_dataloader= torch.utils.data.DataLoader(
    test_dataset, batch_size=2, shuffle =False, num_workers=0)

In [16]:
for (idx,(x,y)) in enumerate(train_dataloader):
    print(f'Batch: #{idx}, \nx: {x}, \ny:{y}')

Batch: #0, 
x: tensor([[143.0000,   5.8000,   2.7000,   5.1000,   1.9000],
        [ 92.0000,   6.1000,   3.0000,   4.6000,   1.4000]]), 
y:tensor([2, 1])
Batch: #1, 
x: tensor([[34.0000,  5.5000,  4.2000,  1.4000,  0.2000],
        [54.0000,  5.5000,  2.3000,  4.0000,  1.3000]]), 
y:tensor([0, 1])
Batch: #2, 
x: tensor([[43.0000,  4.4000,  3.2000,  1.3000,  0.2000],
        [22.0000,  5.1000,  3.7000,  1.5000,  0.4000]]), 
y:tensor([0, 0])
Batch: #3, 
x: tensor([[87.0000,  6.7000,  3.1000,  4.7000,  1.5000],
        [44.0000,  5.0000,  3.5000,  1.6000,  0.6000]]), 
y:tensor([1, 0])
Batch: #4, 
x: tensor([[36.0000,  5.0000,  3.2000,  1.2000,  0.2000],
        [26.0000,  5.0000,  3.0000,  1.6000,  0.2000]]), 
y:tensor([0, 0])
Batch: #5, 
x: tensor([[146.0000,   6.7000,   3.0000,   5.2000,   2.3000],
        [117.0000,   6.5000,   3.0000,   5.5000,   1.8000]]), 
y:tensor([2, 2])
Batch: #6, 
x: tensor([[96.0000,  5.7000,  3.0000,  4.2000,  1.2000],
        [11.0000,  5.4000,  3.7000,  1.5

In [ ]:
ac_old=0
for a in range(0,100):
    model = NeuralNetwork(num_inputs=5, num_outputs=3)
    optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

    num_epochs = 100000
    for epoch in range(num_epochs):
        model.train()
    for (idx,(x,y)) in enumerate(train_dataloader):
        model_result = model(x)
        loss = F.cross_entropy(model_result,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    ac_new=calculate_accuracy_torchmetrics(model,test_dataloader)
    if(ac_new.item()>ac_old):
        best=a
        ac_old=ac_new
ac_old


Точность: 40.00%
Точность: 30.00%
Точность: 50.00%
Точность: 63.33%
Точность: 66.67%
Точность: 66.67%
Точность: 66.67%
Точность: 56.67%
Точность: 66.67%
Точность: 60.00%
Точность: 66.67%
Точность: 66.67%
Точность: 66.67%
Точность: 66.67%
Точность: 60.00%
Точность: 40.00%
Точность: 60.00%
Точность: 50.00%
Точность: 46.67%
Точность: 66.67%
Точность: 66.67%
Точность: 66.67%
Точность: 73.33%
Точность: 73.33%
Точность: 66.67%
Точность: 70.00%
Точность: 53.33%
Точность: 46.67%
Точность: 66.67%
Точность: 46.67%
Точность: 50.00%
Точность: 66.67%
Точность: 66.67%
Точность: 50.00%
Точность: 50.00%
Точность: 50.00%
Точность: 70.00%
Точность: 46.67%
Точность: 76.67%
Точность: 66.67%
Точность: 50.00%
Точность: 66.67%
Точность: 76.67%
Точность: 40.00%
Точность: 66.67%
Точность: 73.33%
Точность: 80.00%
Точность: 73.33%
Точность: 63.33%
Точность: 66.67%
Точность: 63.33%
Точность: 66.67%
Точность: 46.67%
Точность: 46.67%
Точность: 63.33%
Точность: 66.67%
Точность: 50.00%
Точность: 63.33%
Точность: 46.6

tensor(0.8667)

In [18]:
! pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [19]:
from torchmetrics import Accuracy

def calculate_accuracy_torchmetrics(model, dataloader, device='cpu'):
    model.eval()
    accuracy = Accuracy(task='multiclass', num_classes=3).to(device)
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            preds = torch.argmax(outputs, dim=1)
            accuracy.update(preds, y)
    
    acc = accuracy.compute()
    print(f'Точность: {acc * 100:.2f}%')
    return acc

Maximum accuracy 0.6 eto ochen grustno